### Anomaly Detection - Anomaly Scores  Calculations

This notebook shows how to calculate anomaly scores for anomaly detection

In [ ]:
%reload_ext autoreload
%autoreload 2
import tensorflow
import tensorflow as tf
from tensorflow import keras
from keras.utils import plot_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

import IPython, IPython.display, os, datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
mpl.rcParams['figure.figsize'] = (14, 4)
mpl.rcParams['axes.grid'] = True

print(f"Tensorflow Version {tf.__version__}, Keras Vesion: {keras.__version__}")

## Create Utility to compute anomaly score

In [ ]:
#%%writefile "ts_anom_utils.py"

# DO NOT EDIT THIS FILE - GENERATED FROM 06_anom_scores_calc.ipynb

# Study this carefully

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle


'''~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    This will compute the F-score given y and yhat
'''
def computeFScore(y, yhat):
    nu = np.sum( (y - yhat) ** 2)
    de = 1 or np.sum( (y-np.average(y))**2 )

    FSCORE=1-np.sqrt(nu/de)
    return np.round(FSCORE, 4)

'''~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    broken pairs columns are  "x, y, resid, fitness, threshhold, ranking"
    This will return all sensors whose standardized error exceeds threshhold value
'''
def get_brknpairs(r, fscore, standardized_err, errdf, thr=1.4, topn=10):
    r1  = round(r, 4)
    se  =  np.round(abs(standardized_err.loc[r.name]), 4)
    idx = abs(se.values).argsort()

    ret = [[c, c, r1[c], fscore[c], thr, se[c]] for c in se[idx][::-1][:topn].index if abs(se[c]) > thr ]
    top = [[c, se[c]] for c in se[idx][::-1][:topn].index]
    return  len(ret), ret, top

'''~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    compute Anomaly scores
    ydf: original Y Values
    pdf: predicted values

    You must always pass escaler unless you are using validation data to get the estimate of the error

    returns: 
        broken-pairs columns are  "x, y, resid, fitness, threshhold, ranking"
'''
def compute_scores(y: pd.DataFrame, yhat: pd.DataFrame, errorDF= None, escaler=None, top_n=15, file=None): 
    # 1. Compute the Absolute Error data frame
    resid = y - yhat
    error = abs(resid)

    # 2. Lets scale if scaler is not given => assuming this is training errors
    
    if ( escaler is None ):
        escaler = StandardScaler().fit(error)
        standardized_err = pd.DataFrame(escaler.transform(error), columns=error.columns, index=resid.index)

        # Compute the error statistics only if escaler is None
        fscore = computeFScore(y, yhat)

        errorDF = error.describe()
        errorDF.loc['fscore'] = fscore
        errorDF.loc['std_mean'] = standardized_err.mean()
        errorDF.loc['std_std'] = standardized_err.std()
    else:
        standardized_err = pd.DataFrame(escaler.transform(error), columns=error.columns, index=resid.index)


    # 4. Compute the Frobenius norm 
    score = np.linalg.norm(error.values ,axis=1)
    norm_score = np.linalg.norm(standardized_err ,axis=1)

    appl = resid.apply(get_brknpairs, args=( fscore, standardized_err, errdf, 1.4, 20), axis=1)


    # 6. put them all together
    ret = pd.DataFrame(range(len(y)), columns=["line"], index=y.index)
    ret['time']             = y.index
    ret['score']            = score
    ret['norm_score']       = norm_score
    ret['numBroken']        = [c[0] for c  in appl.values]
    ret['brokenInvariants'] = [c[1] for c  in appl.values]
    ret['brokenSensors']    = [c[2] for c  in appl.values]

    if ( file is not None):
        errorDF.to_csv(f'{file}_errordf.csv')
        pickle.dump(escaler, open(f'{file}_escaler.pkl', 'wb'))

    return ret, error, standardized_err, errorDF, escaler, fscore

# The way how to use this 
'''
    Split data into train, validation, and test data
    Train on "training"
    Compute yhat on validation data
    call compute_scores on y and yhat of validation data.
    Use the escaler to detect anomalies on test data
    => COmpute the F-scrore, precision, recall scores
''';




### How to use the utility 

In [ ]:
file="/tmp/"

# Pass file parameter only when you want to save the parameters
ret, error, standardized_err, errorDF, escaler, fscore = compute_scores(y, yhat, file=file)

suffix = str(ret.index[0]).replace(" ", '') + "--" + str(ret.index[-1]).replace(" ", '')
ret.to_csv(f'{file}SCORE-{suffix}.csv', index=False)


# Read back and call for future calls
file="/tmp/test_"
errorDF = pd.read_csv(f'{file}_errordf.csv', index_col=0)
fscore = errorDF.loc['fscore']
escaler= pickle.load(open(f'{file}_escaler.pkl', 'rb') )

ret


## Anomaly score calculations explaination

Explaination of the code above with an exmaple

In [ ]:
# Lets take a sample data and work through anomaly calculations
# We create a 
'''
    ydf => orignal values
    pdf => predicted value
    edf => error of the predictions
'''

np.random.seed(1)

rw = np.array([1,2,3])
ar = np.array([
    rw, rw + 1, rw + 2, rw*2
])
ydf = pd.DataFrame(ar, columns="A B C".split())
ydf.index = pd.date_range("2021-1-1", periods=len(ydf), freq="1H")

# Create a sample prediction data
pdf = pd.DataFrame(ydf.values + np.random.normal(size=ydf.values.shape), columns=ydf.columns)
pdf.index = ydf.index

# Compute the Errr
edf = ydf - pdf
edf.index = ydf.index

abs_edf = abs(edf)
abs_edf.index = ydf.index

sdf=pd.DataFrame(columns=["<->"], data=['<->']*len(ydf))
sdf.index = ydf.index
display(pd.concat([ydf, sdf, pdf, sdf, edf, sdf, abs_edf],  axis=1))

# This will describe your error distribution
edf.describe()

In [ ]:
# Lets compute the scale 
# you will save this errscale for future to validate test scores

errscale = StandardScaler().fit(abs_edf)
standardized_edf = pd.DataFrame( errscale.transform(abs_edf), columns=abs_edf.columns)
standardized_edf

In [ ]:
top_n = 2   # How many top sensors you want -> if you have large data, you may want to choose 10
suspicious_sensors_idx = np.argsort(-standardized_edf.abs().values )[:,:top_n]
suspicious_sensors = [f for f in np.array(standardized_edf.columns)[suspicious_sensors_idx] ]

suspicious_sensors_idx, suspicious_sensors_idx.shape, suspicious_sensors


In [ ]:
# this will give number of sensors deviated by more than by one SD
threshold = 1.1

vals = (standardized_edf.abs() > threshold)
np.count_nonzero(vals, axis=1)

In [ ]:
score = np.linalg.norm(abs_edf.values ,axis=1)
norm_score = np.linalg.norm(standardized_edf.values ,axis=1)
score, norm_score

In [ ]:
# Combine them to get a score you can write 
z = zip(score, norm_score, suspicious_sensors)
adf=pd.DataFrame(z, columns="score norm_score suspicious_sensors".split())

In [ ]:
# Finally create scores file and optinally saVE IT
scoreDF = pd.concat([adf, edf, standardized_edf], axis=1)
#scoreDF.to_csv("/tmp/temp.csv")
#pd.read_csv("/tmp/temp.csv", index_col=0)

In [ ]:
errorDF